In [1]:
import json
from elasticsearch import Elasticsearch
import os
from dotenv import load_dotenv
import minsearch
import requests
import json

# Load environment variables
load_dotenv()

def initialize_elasticsearch():
    """Connect to Elasticsearch"""
    es_client = Elasticsearch('http://localhost:9200', request_timeout=30, retry_on_timeout=True)
    
    # Check if Elasticsearch is running
    try:
        info = es_client.info()
        print(f"Connected to Elasticsearch: {info['version']['number']}")
        return es_client
    except Exception as e:
        print(f"Error connecting to Elasticsearch: {e}")
        return None

def create_index(es_client, index_name="acne-sense-docs"):
    """Create Elasticsearch index with appropriate mappings"""
    # Check if index already exists
    if es_client.indices.exists(index=index_name):
        print(f"Index '{index_name}' already exists")
        return
    
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"}
            }
        }
    }
    
    es_client.indices.create(index=index_name, body=index_settings)
    print(f"Created index '{index_name}'")

def load_documents(file_path="documents.json"):
    """Load documents from JSON file"""
    with open(file_path, 'r', encoding='utf-8') as file:
        documents_raw = json.load(file)

    documents = []
    
    for course in documents_raw:
        course_name = course['course']
        
        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)
    
    return documents

def index_documents(es_client, documents, index_name="acne-sense-docs"):
    """Index documents in Elasticsearch"""
    from tqdm import tqdm
    
    print(f"Indexing {len(documents)} documents...")
    for doc in tqdm(documents):
        es_client.index(index=index_name, document=doc)
    
    print("Indexing complete")

def elastic_search(es_client, query, filter_course=None, index_name="acne-sense-docs", num_results=5):
    """Search documents in Elasticsearch"""
    # Build the search query
    search_query = {
        "size": num_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section^0.5"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    # Add filter if course is specified
    if filter_course:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "course": filter_course
            }
        }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

def build_prompt(query, diagnosis_result, search_results):
    """Build the prompt for the Qwen model"""
    prompt_template = """
You are a dermatology AI assistant for the Acne Sense app. Your role is to provide helpful, accurate information about acne based on the user's skin analysis and questions.

SKIN ANALYSIS RESULT:
{diagnosis}

USER QUESTION: {question}

CONTEXT FROM KNOWLEDGE BASE:
{context}

Provide a helpful, compassionate response that:
1. Acknowledges their specific acne condition
2. Offers relevant treatment suggestions and skincare advice based on their acne type
3. Clearly indicates when professional dermatologist consultation is recommended
4. Uses only information from the provided context
5. Avoids making definitive medical diagnoses or prescribing specific medications
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(diagnosis=diagnosis_result or "No skin analysis provided.", 
                                    question=query,
                                    context=context).strip()
    return prompt

def query_qwen(prompt):
    """Send a query to the Qwen model via Ollama"""
    # Modify this based on your actual Qwen integration method
    # This is an example assuming Ollama API
    url = "http://localhost:11434/api/generate"
    
    # Request payload for Ollama's API
    payload = {
        "model": "qwen2:7b",  # Use qwen2:7b model
        "prompt": prompt,
        "stream": False,      # Don't stream the response
        "options": {
            "temperature": 0.2,
            "top_p": 0.95,
            "top_k": 40,
            "max_tokens": 800
        }
    }
    try:
        # Send the request to Ollama
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise exception for non-2xx responses
        
        # Extract the generated text from the response
        response_data = response.json()
        return response_data["response"]
    
    except requests.exceptions.RequestException as e:
        print(f"Error calling Ollama API: {e}")
        return f"Error generating response: {str(e)}"

def rag(query, diagnosis_result=None, filter_course=None):
    """Run the complete RAG pipeline"""
    # Initialize Elasticsearch client
    es_client = initialize_elasticsearch()
    if not es_client:
        return "Could not connect to Elasticsearch"
    
    # Search for relevant documents
    search_results = elastic_search(es_client, query, filter_course)
    
    # Build prompt with retrieved documents
    prompt = build_prompt(query, diagnosis_result, search_results)
    
    # Query the Qwen model
    answer = query_qwen(prompt)
    
    return answer

# Setup function
def setup_elasticsearch():
    """Initialize and setup Elasticsearch"""
    es_client = initialize_elasticsearch()
    if not es_client:
        return False
    
    create_index(es_client)
    documents = load_documents()
    index_documents(es_client, documents)
    return True


In [2]:
# def query_qwen(prompt):
#     """Send a query to the Qwen model via Ollama using streaming API"""
#     try:
#         response = requests.post(
#             "http://localhost:11434/api/generate",
#             json={
#                 "model": "qwen2.5",
#                 "prompt": prompt,
#                 "temperature": 0.2,
#                 "max_tokens": 800,
#                 "stream": False  # Set to True for streaming, but requires different parsing
#             },
#             stream=False
#         )
        
#         if response.status_code == 200:
#             result = response.json()
#             # For non-streaming mode, Ollama returns a JSON with the field 'response'
#             if "response" in result:
#                 return result["response"]
#             else:
#                 print(f"Unexpected response format: {result}")
#                 return "Could not parse model response."
#         else:
#             print(f"API Error: {response.status_code}")
#             return f"API Error: {response.status_code}"
            
#     except Exception as e:
#         print(f"Error querying Qwen model: {str(e)}")
#         return "An error occurred while processing your request."

In [3]:
setup_successful = setup_elasticsearch()

Connected to Elasticsearch: 8.4.3


C:\Users\Andakara\AppData\Local\Temp\ipykernel_30852\3454098828.py:47: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


Created index 'acne-sense-docs'
Indexing 33 documents...


100%|██████████| 33/33 [00:00<00:00, 83.15it/s]

Indexing complete


In [4]:
if setup_successful:
    print("Setup successful, testing with a query...")
    test_query = "What should I do about papules on my face?"
    test_diagnosis = """
Acne Type: Mild inflammatory acne
Main Lesions Detected:
- Several papules (6)
- Few pustules (2)
Location: Primarily on forehead and cheeks
"""
    result = rag(test_query, test_diagnosis, filter_course="acne-sense-knowledge-base")
    print(f"Question: {test_query}")
    print(f"Answer: {result}")

Setup successful, testing with a query...
Connected to Elasticsearch: 8.4.3


C:\Users\Andakara\AppData\Local\Temp\ipykernel_30852\3454098828.py:102: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


Question: What should I do about papules on my face?
Answer: Hello! I understand you're dealing with mild inflammatory acne characterized by several papules (6) and a few pustules (2), primarily on your forehead and cheeks. This is considered mild, but it's important to manage properly to prevent worsening of symptoms.

For the papules you have, which are small, solid pimples that can be pink or red and tender to touch, here’s what I recommend:

1. **Avoid Picking**: It's crucial not to pick or squeeze these lesions as this can worsen inflammation and increase the risk of scarring.
2. **Topical Treatments**: Consider using topical treatments containing retinoids, benzoyl peroxide, or salicylic acid. These are effective in treating papular acne by reducing inflammation and unclogging pores. Apply them gently to your affected areas as directed.
3. **Skincare Routine**: Incorporate a gentle cleanser into your daily routine, followed by a non-comedogenic moisturizer that doesn't clog pores